In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [5]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan


def add_beep_boop(row):
  macdhist, ema50, mid_low, mid_high, rsi = row[['macdhist', 'ema50', 'Mid_Low', 'Mid_High', 'rsi']]

  if float(macdhist) > 0 and float(mid_low) > float(ema50):
    return 1

  elif float(macdhist) < 0 and float(mid_high) < float(ema50):
    return 2

  else:
    return 0

In [6]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['beep_boop'] = df.apply(add_beep_boop, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['beep_boop'].value_counts())
print(df['fractal'].value_counts())

0    35299
1    20431
2    18333
Name: beep_boop, dtype: int64
0.0    55662
1.0     9387
2.0     9014
Name: fractal, dtype: int64


In [7]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [10]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def get_end_date(j):
  return df.loc[df.index[j + 1], 'Date'] if j + 1 < len(df) else None

def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trade = None

    i = 2

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']
        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        prev_mo = df.loc[df.index[i - 1], 'Mid_Open']
        prev_mc = df.loc[df.index[i - 1], 'Mid_Close']
        curr_ml2 = df.loc[df.index[i - 2], 'Mid_Low']
        curr_ml1 = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh2 = df.loc[df.index[i - 2], 'Mid_High']
        curr_mh1 = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh1 - curr_ml1)
        beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
        beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
        trending = adx >= adx_cutoff
        curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
          df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                              'Ask_Low', 'Ask_Close']]

        if trade is None:
          if beep_boop2 == 1 and beep_boop1 == 1 and curr_ml2 > ema200 and curr_ml1 > ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_buys = True

              if all_buys:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 1:
                        curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                        curr_fractal_distance = curr_ao - curr_pullback

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_ask_open)
                    stop_loss = round(pullback - pullback_cushion, 5)
                    # stop_loss = round(pullback - pullback_cushion, 3)

                    if stop_loss < open_price:
                        curr_pips_to_risk = open_price - stop_loss

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                              'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                              'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_buys += 1

                            pips_risked.append(curr_pips_to_risk)

          elif beep_boop2 == 2 and beep_boop1 == 2 and curr_mh2 < ema200 and curr_mh1 < ema200 and enough_volatility and trending and curr_bar_length <= bar_length:
              all_sells = True

              if all_sells:
                pullback = None
                u = i - 3

                while u >= 0:
                    if df.loc[df.index[u], 'fractal'] == 2:
                        curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                        curr_fractal_distance = curr_pullback - curr_bo

                        if curr_fractal_distance >= fractal_distance:
                            pullback = curr_pullback
                            break

                    u -= 1

                if pullback is not None:
                    open_price = float(curr_bid_open)
                    stop_loss = round(pullback + pullback_cushion, 5)
                    # stop_loss = round(pullback + pullback_cushion, 3)

                    if stop_loss > open_price:
                        curr_pips_to_risk = stop_loss - open_price

                        if curr_pips_to_risk <= max_pips_to_risk:
                            stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                            # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                            trade = {'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                    'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                    'n_units': n_units, 'start_date': curr_date, 'end_date': None}

                            n_sells += 1

                            pips_risked.append(curr_pips_to_risk)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
          trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
            trade['stop_loss'] = curr_bid_high - trade['pips_risked']

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
          trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
          trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        if trade is not None and trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
          trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
          reward += trade_amount
          day_fees += calculate_day_fees(trade['start_date'], curr_date)

          n_wins += 1 if trade_amount > 0 else 0
          n_losses += 1 if trade_amount < 0 else 0
          curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
          curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

          if curr_win_streak > win_streak:
            win_streak = curr_win_streak

          if curr_loss_streak > loss_streak:
            loss_streak = curr_loss_streak

          trade = None

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1, 1.5, 2]
# risk_reward_ratio_vals = [2]
pullback_cushion_vals = [0, 5]
# pullback_cushion_vals = [5]
fractal_distances = [5, 10, 15, 20]
# fractal_distances = [15]
bar_lengths = [5, 10, 50]
# bar_lengths = [5]
adx_cutoffs = [15, 20, 25]
# adx_cutoffs = [20]
spread_cutoffs = [2, 3]
# spread_cutoffs = [2]
n_possibilities = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(fractal_distances) * len(bar_lengths) * len(adx_cutoffs) * len(spread_cutoffs)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for fractal_distance in fractal_distances:
      for bar_length in bar_lengths:
        for adx_cutoff in adx_cutoffs:
          for spread_cutoff in spread_cutoffs:
            all_combos.append((risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_pullback_cushion = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_adx_cutoff = None
best_spread_cutoff = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or reward > min_item['reward']:
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'pullback': pullback_cushion, 'fractal_dist': fractal_distance, 'bar_length': bar_length, 'adx': adx_cutoff, 'spread': spread_cutoff})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_pullback_cushion = pullback_cushion
    best_fractal_distance = fractal_distance
    best_bar_length = bar_length
    best_adx_cutoff = adx_cutoff
    best_spread_cutoff = spread_cutoff

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 432

-16219.999999999953
Num buys: 564
Num sells: 627
Num trades: 1191
Num wins: 372
Num losses: 818
Win streak: 6
Loss streak: 14
Remaining runs: 431
Best reward so far: -16219.999999999953

-17906.999999999884
Num buys: 619
Num sells: 630
Num trades: 1249
Num wins: 400
Num losses: 847
Win streak: 6
Loss streak: 12
Remaining runs: 430
Best reward so far: -16219.999999999953

-6892.000000000121
Num buys: 414
Num sells: 453
Num trades: 867
Num wins: 309
Num losses: 557
Win streak: 5
Loss streak: 10
Remaining runs: 429
Best reward so far: -6892.000000000121

-6249.499999999928
Num buys: 337
Num sells: 365
Num trades: 702
Num wins: 237
Num losses: 464
Win streak: 5
Loss streak: 11
Remaining runs: 428
Best reward so far: -6249.499999999928

-7429.000000000085
Num buys: 460
Num sells: 507
Num trades: 967
Num wins: 347
Num losses: 619
Win streak: 6
Loss streak: 10
Remaining runs: 427
Best reward so far: -6249.499999999928

-3971.999999999896
Num buys: 351
Num sells: 371
Num trades:

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best fractal distance: ' + str(best_fractal_distance))
print('Best bar length: ' + str(best_bar_length))
print('Best spread: ' + str(best_spread_cutoff))
print('Best adx: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')
for entry in best_rewards:
    print(entry)

# 2020-2021:


------------ FINAL RESULTS ------------
Best reward: 8885.499999999864
Best risk/reward ratio: 2
Best pullback cushion: 15
Best fractal distance: 15
Best bar length: 10
Best spread: 4
Best adx: 20
-----------------------
Top results:
{'reward': 7376, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
{'reward': 7760, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
{'reward': 6834, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
{'reward': 6510, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
{'reward': 5412, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
{'reward': 8885, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
{'reward': 6626, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 3}
{'reward': 7357, 'rati

In [ ]:
import json

set1 = { json.dumps({1: 'hi', 2: 'hello'}, sort_keys=True), json.dumps({3: 'foo', 4: 'foo2'}, sort_keys=True) }
set2 = { json.dumps({1: 'hi', 2: 'hello'}, sort_keys=True) }

set1.intersection(set2)

In [17]:
def add_spread(row):
  curr_ao, curr_bo = row[['Ask_Open', 'Bid_Open']]

  return abs(curr_ao - curr_bo)

df_small['spread'] = df_small.apply(add_spread, axis=1)

In [19]:
df_small['spread'].mean()

0.00020055720133577672

In [20]:
df_small['adx'] = talib.ADX(df_small['Mid_High'], df_small['Mid_Low'], df_small['Mid_Close'], timeperiod=14)

In [21]:
df_small['adx'].mean()

24.594419713453895

In [22]:
foo = pd.read_csv(file_path + 'Oanda_Usd_Jpy_M5_2020-2021.csv')
foo.Date = pd.to_datetime(foo.Date)
foo.reset_index(drop=True, inplace=True)
foo['spread'] = foo.apply(add_spread, axis=1)
foo['adx'] = talib.ADX(foo['Mid_High'], foo['Mid_Low'], foo['Mid_Close'], timeperiod=14)

In [23]:
foo['spread'].mean()

0.014943014185830442

In [24]:
foo['adx'].mean()

24.84253478202672